# **Laboratorio 11: LLM y Agentes Autónomos 🤖**

MDS7202: Laboratorio de Programación Científica para Ciencia de Datos

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### **Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados**

- Nombre de alumno 1: Nicolas Herrera
- Nombre de alumno 2: Lucas Carrasco

### **Link de repositorio de GitHub:** [Repositorio](https://github.com/vspartamo/MDS7202)

## **Temas a tratar**

- Reinforcement Learning
- Large Language Models

## **Reglas:**

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### **Objetivos principales del laboratorio**

- Resolución de problemas secuenciales usando Reinforcement Learning
- Habilitar un Chatbot para entregar respuestas útiles usando Large Language Models.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

## **1. Reinforcement Learning (2.0 puntos)**

En esta sección van a usar métodos de RL para resolver dos problemas interesantes: `Blackjack` y `LunarLander`.

In [1]:
!pip install -qqq gymnasium stable_baselines3
!pip install -qqq swig
!pip install -qqq gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### **1.1 Blackjack (1.0 puntos)**

<p align="center">
  <img src="https://www.recreoviral.com/wp-content/uploads/2016/08/s3.amazonaws.com-Math.gif"
" width="400">
</p>

La idea de esta subsección es que puedan implementar métodos de RL y así generar una estrategia para jugar el clásico juego Blackjack y de paso puedan ~~hacerse millonarios~~ aprender a resolver problemas mediante RL.

Comencemos primero preparando el ambiente. El siguiente bloque de código transforma las observaciones del ambiente a `np.array`:


In [2]:
import gymnasium as gym
from gymnasium.spaces import MultiDiscrete
import numpy as np

In [3]:
class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservation, self).__init__(env)
        self.observation_space = MultiDiscrete(np.array([32, 11, 2]))

    def observation(self, observation):
        return np.array(observation).flatten()

# Create and wrap the environment
env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

#### **1.1.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas.

`escriba su respuesta acá`

El ambiente Blackjack simula el conocido juego de cartas, formulado como un proceso de decisión de Markov (MDP). Primero se dará una breve descripción del juego y luego sus componentes como MDP:

El objetivo del BlackJack es obtener una mano de cartas cuya suma sea lo más cercana posible a 21 sin pasarse. Sus reglas son:

1. **Cartas y valores**:
    - Las cartas numeradas del 2 al 10 tienen su valor nominal.
    - Las cartas con figuras (J, Q, K) valen 10 puntos cada una.
    - El As puede valer 1 u 11 puntos, dependiendo de cuál valor beneficie más al jugador.

2. **El juego**:
    - Cada jugador recibe dos cartas al inicio del juego, al igual que el dealer (la casa). Una de las cartas del dealer se muestra boca arriba y la otra boca abajo.
    - Los jugadores pueden optar por "pedir" (hit) más cartas para acercarse a 21 o "plantarse" (stand) si creen que su mano es lo suficientemente fuerte.
    - Si un jugador supera los 21 puntos, pierde automáticamente (esto se llama "bust").
    - Después de que todos los jugadores han terminado sus turnos, el dealer revela su carta oculta y debe seguir ciertas reglas: debe pedir cartas hasta alcanzar al menos 17 puntos y plantarse si tiene 17 o más.

3. **Ganador**:
    - Si el jugador tiene una mano más cercana a 21 que la del dealer sin pasarse, gana.
    - Si el dealer supera los 21 puntos, todos los jugadores que no se hayan pasado ganan.
    - Si el jugador y el dealer tienen la misma cantidad de puntos, es un empate (push) y se devuelve la apuesta al jugador.

Descripción como MDP:

**Estados**
El espacio de estados es un tupla que describe:
1. **Suma actual del jugador**: Rango [4, 21] (sin superar 21).
2. **Valor de la carta visible del dealer**: Rango [1, 10], donde 1 representa un As.
3. **As utilizable**: Binario (0 o 1), indicando si el jugador tiene un As que puede contar como 11 sin pasarse de 21.

Estado: `(player_sum, dealer_card, usable_ace)`

**Acciones**
El jugador tiene dos acciones discretas:
- `0`: Stick (plantarse, no pedir más cartas).
- `1`: Hit (pedir otra carta).

Espacio de acciones: `Discrete(2)`

**Recompensas**
- `+1`: Si el jugador gana (sin blackjack natural).
- `-1`: Si el jugador pierde.
- `0`: Si hay empate (draw).
- `+1.5`: Si el jugador gana con un blackjack natural (opcional según el parámetro `natural`).
- Nota: Si el jugador excede 21, pierde inmediatamente (`-1`).

**Dinámica del MDP**
- Transición: Las acciones determinan si el jugador toma otra carta o se planta. Si toma otra carta y excede 21, el episodio termina.
- El dealer sigue una estrategia fija: revela su carta oculta y continúa sacando hasta alcanzar un mínimo de 17.
- Los episodios terminan cuando el jugador se planta, excede 21, o el dealer determina el resultado.

#### **1.1.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 5000 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política? ¿Cómo podría interpretar las recompensas obtenidas?

In [3]:
n_episodes = 5000
rewards = []

for _ in range(n_episodes):
    observation = env.reset()
    done = False
    total_reward = 0

    while not done:
        action = env.action_space.sample() #accion random
        observation, reward, done, _, _ = env.step(action)
        total_reward += reward

    rewards.append(total_reward)

print(f"Promedio de las recompensas: {np.mean(rewards):.4f}")
print(f"Desviación estándar de las recompensas: {np.std(rewards):.4f}")

Promedio de las recompensas: -0.3628
Desviación estándar de las recompensas: 0.9088


Un promedio de recompensa de -0.3628 dice que jugar al azar tiende levemente a la perdida, hace sentido pues si no se tiene en consideración la suma actual de las cartas y se pide al azar, pareciera ser más probable pasarse de 21, además, la desviación estándar de 0.9088 dice que hay una alta variabilidad en los resultados, por lo que las recompensas obtenidad no parecieran tener alguna tendencia muy definida, por lo que no es una buena manera de maximizar las recompensas.


#### **1.1.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `Blackjack`.

In [8]:
import gymnasium as gym
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env
from gymnasium.spaces import MultiDiscrete
import numpy as np

In [5]:
class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservation, self).__init__(env)
        self.observation_space = MultiDiscrete(np.array([32, 11, 2]))

    def observation(self, observation):
        return np.array(observation).flatten()

env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

vec_env = make_vec_env(lambda: env, n_envs=1)

model = A2C("MlpPolicy", vec_env, verbose=1) #se escoge A2C

model.learn(total_timesteps=100000)

model.save("a2c_blackjack_model")

episodes = 100
rewards = []
for _ in range(episodes):
    obs = vec_env.reset()
    done = False
    total_reward = 0

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, _ = vec_env.step(action)
        total_reward += reward

    rewards.append(total_reward)

print(f"Promedio de recompensa después del entrenamiento: {np.mean(rewards):.4f}")
print(f"Desviación estándar de recompensa: {np.std(rewards):.4f}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Using cpu device
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.07     |
|    ep_rew_mean        | -0.26    |
| time/                 |          |
|    fps                | 376      |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.131   |
|    explained_variance | 0.509    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 0.0137   |
|    value_loss         | 0.54     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.02     |
|    ep_rew_mean        | -0.1     |
| time/                 |          |
|    fps                | 375      |
|    iterations         | 200      |
|    time_elapsed       | 2        |
|    total_timesteps    | 1000     |
| train/             

#### **1.1.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.1.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [6]:
episodes = 5000
rewards = []

for _ in range(episodes):
    obs = vec_env.reset()
    done = False
    total_reward = 0

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, _= vec_env.step(action)
        total_reward += reward

    rewards.append(total_reward)

print(f"Promedio de las recompensas: {np.mean(rewards):.4f}")
print(f"Desviación estándar de las recompensas: {np.std(rewards):.4f}")


Promedio de las recompensas: -0.1092
Desviación estándar de las recompensas: 0.9483


Como era de esperar, el promedio de recompensas aumentó, pero lo que no era tan predecible es que la desviación estándar no disminuyera, por lo que se tiene que el modelo no alcanza una estrategia consistente en sus resultados. De este tenemos que hay mejores formas de jugar que al azar, pero no está claro si haya una forma de consistentemente ganar.


#### **1.1.5 Estudio de acciones (0.2 puntos)**

Genere una función que reciba un estado y retorne la accion del agente. Luego, use esta función para entregar la acción escogida frente a los siguientes escenarios:

- Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as
- Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as

¿Son coherentes sus acciones con las reglas del juego?

Hint: ¿A que clase de python pertenecen los estados? Pruebe a usar el método `.reset` para saberlo.

In [7]:
def get_action(agent, state):
    """
    Función para obtener la acción del agente dado un estado específico.

    Args:
        agent: Modelo entrenado (e.g., A2C).
        state: Estado del ambiente (tuple con la suma del jugador, carta del dealer y si tiene un as usable).

    Returns:
        Acción (0: Stick, 1: Hit).
    """
    state = np.array(state).reshape(1, -1)
    action, _ = agent.predict(state, deterministic=True)
    return action[0]

scenario_1 = (6, 7, 0)   # suma del agente: 6, dealer muestra: 7, no tiene as
scenario_2 = (19, 3, 1)  # suma del agente: 19, dealer muestra: 3, tiene un as usable

action_1 = get_action(model, scenario_1)
action_2 = get_action(model, scenario_2)

print(f"Acción para el escenario 1: {action_1} (0: Stick, 1: Hit)")
print(f"Acción para el escenario 2: {action_2} (0: Stick, 1: Hit)")


Acción para el escenario 1: 1 (0: Stick, 1: Hit)
Acción para el escenario 2: 0 (0: Stick, 1: Hit)


Parecen ser razonables, veamos cada acción:

1) Estado: Agente tiene una suma baja (6), mientras que el dealer muestra un 7.
En este caso lo esperado esq el agente debería pedir (hit, 1) para intentar mejorar su suma, ya que pararse (stick, 0) con 6 sería casi seguro una pérdida.

2) Estado: Agente tiene 19 con un as usable, y el dealer muestra un 3.
Se espera que el agente se quedara (stick, 0), ya que 19 es un puntaje fuerte y arriesgarse con un hit podría llevarlo a pasarse.

### **1.2 LunarLander**

<p align="center">
  <img src="https://i.redd.it/097t6tk29zf51.jpg"
" width="400">
</p>

Similar a la sección 2.1, en esta sección usted se encargará de implementar una gente de RL que pueda resolver el ambiente `LunarLander`.

Comencemos preparando el ambiente:


In [4]:
import gymnasium as gym
env = gym.make("LunarLander-v3", render_mode = "rgb_array", continuous = True) # notar el parámetro continuous = True

Noten que se especifica el parámetro `continuous = True`. ¿Que implicancias tiene esto sobre el ambiente?

Además, se le facilita la función `export_gif` para el ejercicio 2.2.4:

In [34]:
import imageio
import numpy as np

def export_gif(model, n = 5):
  '''
  función que exporta a gif el comportamiento del agente en n episodios
  '''
  images = []
  for episode in range(n):
    obs = model.env.reset()
    img = model.env.render()
    done = False
    while not done:
      images.append(img)
      action, _ = model.predict(obs)
      obs, reward, done, info = model.env.step(action)
      img = model.env.render(mode="rgb_array")

  imageio.mimsave("agent_performance.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

#### **1.2.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [LunarLander](https://gymnasium.farama.org/environments/box2d/lunar_lander/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas. ¿Como se distinguen las acciones de este ambiente en comparación a `Blackjack`?

Nota: recuerde que se especificó el parámetro `continuous = True`

`escriba su respuesta acá`

El ambiente **LunarLander** modela el problema de optimización de trayectorias de un cohete para aterrizar en una plataforma. Está formulado como un MDP:

- **Estados**:  
  El espacio de observación es continuo, representado por un vector de 8 dimensiones:  
  1. Coordenadas `x` e `y` del cohete.  
  2. Velocidades lineales en `x` e `y`.  
  3. Ángulo y velocidad angular.  
  4. Dos booleanos indicando si las patas del cohete están en contacto con el suelo.  

- **Acciones**:  
  Con el parámetro `continuous=True`, el espacio de acciones es continuo y consiste en un vector de dos dimensiones:  
  1. Intensidad del motor principal (valor entre -1 y 1).  
  2. Intensidad de los propulsores laterales (valor entre -1 y 1).  

  Las acciones determinan el uso de los motores:  
  - Motor principal solo funciona con al menos el 50% de potencia.  
  - Propulsores laterales no se activan entre valores de -0.5 y 0.5.  

- **Recompensas**:  
  Recompensas se otorgan según:  
  - Cercanía del cohete a la plataforma.  
  - Reducción de velocidad (lineal y angular).  
  - Estabilidad en el ángulo.  
  - Contacto de las patas con el suelo (+10 por cada pata).  
  - Penalizaciones por el uso de motores (0.03 por cada frame para propulsores laterales y 0.3 para el motor principal).  
  - Recompensas grandes (+100 por aterrizaje exitoso, -100 por accidente).  

- **Término**:  
  El episodio termina si:  
  - El cohete se estrella.  
  - Sale del área visible.  
  - Aterriza con éxito y se estabiliza.  


Notemos que a diferencia de BlackJack, en este caso se tiene que el espacio de estados es diferente ya que en el primer caso, el BlackJack tiene una combinación finita de cartas y estados (i.e espacio discreto) mientras que LunarLander cuenta con un espacio continuo de 8 dimensiones. Por otro lado, el espacio de acciones es diferente en cada caso, en BlackJack el espacio de acciones es discreto y cuenta con dos posibilidades, pararse o pedir, LunarLander tiene un espacio de acciones continuo bidimensional para regular la potencia de los motores. Por último, las recompensas son distintas ya que en el BlackJack las recompensas se entregan únicamente al final del episodio y en LunarLander se entregan acumulativamente, dándose en cada paso del episodio según el desempeño.

#### **1.2.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 10 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política?

In [6]:
num_episodes = 10
rewards = []

# con acciones aleatorias
for episode in range(num_episodes):
    state = env.reset()[0]
    done = False
    total_reward = 0

    while not done:
        action = env.action_space.sample()  # acción al azar
        state, reward, done, _, _ = env.step(action)
        total_reward += reward

    rewards.append(total_reward)

print(f"Promedio de recompensas: {np.mean(rewards):.2f}")
print(f"Desviación estándar de recompensas: {np.std(rewards):.2f}")

Promedio de recompensas: -240.96
Desviación estándar de recompensas: 99.86


Esta estrategia es muy mala, la recompensa promedio negativa indica que en general se llega a acciodentes o que se hizo muchas cosas negativas para lo buscado, el cohete en la mayoría de casos no aterriza y en general la recompensa final es negativa, ya que pese a que la desviación estándar es alta, no es lo suficiente como para que esta estrategia se acerque consistentemente a resultados de recompensa positiva.

#### **1.2.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `LunarLander` **usando 10000 timesteps de entrenamiento**.

In [9]:
env = gym.make("LunarLander-v3", continuous=True)

model = A2C("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

# Guardar el modelo entrenado
model.save("a2c_lunar_lander")


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run A2C on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


------------------------------------
| rollout/              |          |
|    ep_len_mean        | 115      |
|    ep_rew_mean        | -219     |
| time/                 |          |
|    fps                | 219      |
|    iterations         | 100      |
|    time_elapsed       | 2        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -2.85    |
|    explained_variance | -0.00148 |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 26.8     |
|    std                | 1.01     |
|    value_loss         | 80.4     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 145      |
|    ep_rew_mean        | -342     |
| time/                 |          |
|    fps                | 297      |
|    iterations         | 200      |
|    time_elapsed       | 3        |
|    total_timesteps    | 1000     |
|

#### **1.2.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.2.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [10]:
model = A2C.load("a2c_lunar_lander")

rewards = []
for episode in range(10):
    obs = env.reset()[0]
    episode_reward = 0
    done = False
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, _, _ = env.step(action)
        episode_reward += reward
    rewards.append(episode_reward)

print(f"Promedio de recompensas: {np.mean(rewards)}")
print(f"Desviación estándar de recompensas: {np.std(rewards)}")


Promedio de recompensas: -448.96739943018366
Desviación estándar de recompensas: 123.7272590595172


Dada la naturaleza del problema, lo esperable es que mejoraran los resultados ya que se puede pensar que existen estrategias ganadoras, y en consecuencia, que el modelo sea capaz de encontrar alguna, esto quizás se pueda alcanzar con un alto número de iteraciones pero también aumenta mucho el tiempo de cómputo, por lo que al quedarnos con un modelo de 10000 iteraciones, se obtiene resultados similares a escoger las acciones al azar, de hecho, con este modelo la desviación estándar aumenta, lo que hace que sea peor que el baseline, ya que la estrategia obtenida no muestra resultados consistentes.

#### **1.2.5 Optimización de modelo (0.2 puntos)**

Repita los ejercicios 1.2.3 y 1.2.4 hasta obtener un nivel de recompensas promedio mayor a 50. Para esto, puede cambiar manualmente parámetros como:
- `total_timesteps`
- `learning_rate`
- `batch_size`

Una vez optimizado el modelo, use la función `export_gif` para estudiar el comportamiento de su agente en la resolución del ambiente y comente sobre sus resultados.

Adjunte el gif generado en su entrega (mejor aún si además adjuntan el gif en el markdown).

In [30]:
from stable_baselines3.common.callbacks import EvalCallback

env = make_vec_env("LunarLander-v3", n_envs=1)

model1 = A2C(
    "MlpPolicy",
    env,
    learning_rate=0.005,
    verbose=1,
)
model1.learn(total_timesteps=10000)  # aumentar timesteps

Using cuda device


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run A2C on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


------------------------------------
| rollout/              |          |
|    ep_len_mean        | 88.8     |
|    ep_rew_mean        | -397     |
| time/                 |          |
|    fps                | 358      |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.01    |
|    explained_variance | 0.074    |
|    learning_rate      | 0.005    |
|    n_updates          | 99       |
|    policy_loss        | 0.911    |
|    value_loss         | 5.07     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 124      |
|    ep_rew_mean        | -496     |
| time/                 |          |
|    fps                | 314      |
|    iterations         | 200      |
|    time_elapsed       | 3        |
|    total_timesteps    | 1000     |
| train/                |          |
|

In [31]:
rewards = []
for episode in range(10):
    obs = env.reset()
    episode_reward = 0
    done = False
    while not done:
        action, _ = model1.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
        episode_reward += reward[0]
        done = done[0]
    rewards.append(episode_reward)

print(f"Promedio de recompensas optimizado: {np.mean(rewards)}")
print(f"Desviación estándar optimizado: {np.std(rewards)}")

Promedio de recompensas optimizado: 54.099287352489775
Desviación estándar optimizado: 106.20646700784887


In [33]:
export_gif(model1)

## **2. Large Language Models (4.0 puntos)**

En esta sección se enfocarán en habilitar un Chatbot que nos permita responder preguntas útiles a través de LLMs.

### **2.0 Configuración Inicial**

<p align="center">
  <img src="https://media1.tenor.com/m/uqAs9atZH58AAAAd/config-config-issue.gif"
" width="400">
</p>

Como siempre, cargamos todas nuestras API KEY al entorno:

In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, max_retries=2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002025B731F40>, default_metadata=())

### **2.1 Retrieval Augmented Generation (1.5 puntos)**

<p align="center">
  <img src="https://y.yarn.co/218aaa02-c47e-4ec9-b1c9-07792a06a88f_text.gif"
" width="400">
</p>

El objetivo de esta subsección es que habiliten un chatbot que pueda responder preguntas usando información contenida en documentos PDF a través de **Retrieval Augmented Generation.**

#### **2.1.1 Reunir Documentos (0 puntos)**

Reuna documentos PDF sobre los que hacer preguntas siguiendo las siguientes instrucciones:
  - 2 documentos .pdf como mínimo.
  - 50 páginas de contenido como mínimo entre todos los documentos.
  - Ideas para documentos: Documentos relacionados a temas académicos, laborales o de ocio. Aprovechen este ejercicio para construir algo útil y/o relevante para ustedes!
  - Deben ocupar documentos reales, no pueden utilizar los mismos de la clase.
  - Deben registrar sus documentos en la siguiente [planilla](https://docs.google.com/spreadsheets/d/1Hy1w_dOiG2UCHJ8muyxhdKPZEPrrL7BNHm6E90imIIM/edit?usp=sharing). **NO PUEDEN USAR LOS MISMOS DOCUMENTOS QUE OTRO GRUPO**
  - **Recuerden adjuntar los documentos en su entrega**.

In [ ]:
%pip install --upgrade --quiet PyPDF2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import PyPDF2

doc_paths = ['AttentionIsAllYouNeed.pdf', 'BERT.pdf', 'ChainOfThought.pdf'] # rellenar con los path a sus documentos

assert len(doc_paths) >= 2, "Deben adjuntar un mínimo de 2 documentos"

total_paginas = sum(len(PyPDF2.PdfReader(open(doc, "rb")).pages) for doc in doc_paths)
assert total_paginas >= 50, f"Páginas insuficientes: {total_paginas}"

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

docs = []
for path in doc_paths:
    loader = PyPDFLoader(path)
    docs += loader.load()

docs[:5]

[Document(metadata={'source': 'AttentionIsAllYouNeed.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architectu

#### **2.1.2 Vectorizar Documentos (0.2 puntos)**

Vectorice los documentos y almacene sus representaciones de manera acorde.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)
splits[:5]

[Document(metadata={'source': 'AttentionIsAllYouNeed.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu'),
 Document(metadata={'source': 'AttentionIsAllYouNeed.pdf', 'page': 0}, page_content='University of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)
vectorstore

#### **2.1.3 Habilitar RAG (0.3 puntos)**

Habilite la solución RAG a través de una *chain* y guárdela en una variable.

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever_chain = retriever | format_docs

#### **2.1.4 Verificación de respuestas (0.5 puntos)**

Genere un listado de 3 tuplas ("pregunta", "respuesta correcta") y analice la respuesta de su solución para cada una. ¿Su solución RAG entrega las respuestas que esperaba?

Ejemplo de tupla:
- Pregunta: ¿Quién es el presidente de Chile?
- Respuesta correcta: El presidente de Chile es Gabriel Boric

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# noten como ahora existe el parámetro de context!
rag_template = '''
Eres un asistente experto en investigación sobre LLMs.
Tu único rol es contestar preguntas del usuario a partir de información relevante que te sea proporcionada.
Responde siempre de la forma más completa posible y usando toda la información entregada.
Responde sólo lo que te pregunten a partir de la información relevante, NUNCA inventes una respuesta.

Información relevante: {context}
Pregunta: {question}
Respuesta:
'''

rag_prompt = PromptTemplate.from_template(rag_template)

rag_chain = (
    {
        "context": retriever_chain,
        "question": RunnablePassthrough(),
    } # Los parámetros de la plantilla
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question_answer_list = [
    (
        "Qué es un Transformer?",
        "Un Transformer es un tipo de arquitectura de red neuronal que usa un encoder-decoder para modelar relaciones en datos secuenciales."
    ),
    (
        "Según Chain of Thought, ¿qué técnica se puede usar para resolver un problema complejo con un LLM?",
        "Se puede descomponer el problema en partes más pequeñas y resolver cada parte por separado."
    )
]

for question_answer in question_answer_list:
    res = rag_chain.invoke(question_answer[0])
    print("Respuesta esperada: ", question_answer[1])
    print("Respuesta obtenita: ", res)
    print()

Respuesta esperada:  Un Transformer es un tipo de arquitectura de red neuronal que usa un encoder-decoder para modelar relaciones en datos secuenciales.
Respuesta obtenita:  Un Transformer es un modelo de transducción que utiliza únicamente auto-atención para calcular representaciones de su entrada y salida, sin usar RNNs alineados con la secuencia o convoluciones.  Su arquitectura se compone de un codificador y un decodificador, ambos formados por una pila de N=6 capas idénticas. Cada capa tiene dos subcapas: un mecanismo de auto-atención multi-cabeza y una capa de conexión completamente conectada posicional.  El modelo utiliza codificaciones posicionales tanto en el codificador como en el decodificador, y un dropout de Pdrop = 0.1 en el modelo base. Durante el entrenamiento, se empleó suavizado de etiquetas con un valor de ϵls = 0.1.


Respuesta esperada:  Se puede descomponer el problema en partes más pequeñas y resolver cada parte por separado.
Respuesta obtenita:  Según la informa

**RESPUESTA**: La solución de RAG entrega respuestas correctas, y bastante parecidas a las esperadas. Al ser preguntas no simples, era difícil que la respuesta fuera exactamente igual, pero en general, la respuesta entregada por RAG es correcta.

#### **2.1.5 Sensibilidad de Hiperparámetros (0.5 puntos)**

Extienda el análisis del punto 2.1.4 analizando cómo cambian las respuestas entregadas cambiando los siguientes hiperparámetros:
- `Tamaño del chunk`. (*¿Cómo repercute que los chunks sean mas grandes o chicos?*)
- `La cantidad de chunks recuperados`. (*¿Qué pasa si se devuelven muchos/pocos chunks?*)
- `El tipo de búsqueda`. (*¿Cómo afecta el tipo de búsqueda a las respuestas de mi RAG?*)

In [ ]:
def try_other_config(
    chunk_size: int = 500,
    ammount_of_chunks: int = 3,
    search_type: str = 'similarity'
):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=50)
    splits = text_splitter.split_documents(docs)

    embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # inicializamos los embeddings
    vectorstore = FAISS.from_documents(documents=splits, embedding=embedding) # vectorizacion y almacenamiento

    retriever = vectorstore.as_retriever(
        search_type=search_type, # método de búsqueda
        search_kwargs={"k": ammount_of_chunks}, # n° documentos a recuperar
    )

    retriever_chain = retriever | format_docs

    rag_chain = (
        {
            "context": retriever_chain,
            "question": RunnablePassthrough(),
        }
        | rag_prompt
        | llm
        | StrOutputParser()
    )

    for question_answer in question_answer_list:
        res = rag_chain.invoke(question_answer[0])
        print("Respuesta esperada: ", question_answer[1])
        print("Respuesta obtenita: ", res)
        print()


**PROBANDO CAMBIAR TAMAÑO DE CHUNK**

In [ ]:
try_other_config(chunk_size=50)

Respuesta esperada:  Un Transformer es un tipo de arquitectura de red neuronal que usa un encoder-decoder para modelar relaciones en datos secuenciales.
Respuesta obtenita:  Basado en la información proporcionada, un Transformer es un modelo que puede ser usado para tareas de traducción.  También se menciona que existe una versión del Transformer referida como "Transformer".


Respuesta esperada:  Se puede descomponer el problema en partes más pequeñas y resolver cada parte por separado.
Respuesta obtenita:  Según la información proporcionada, la técnica "chain of thought" es particularmente adecuada para resolver problemas complejos con un LLM.  El ejemplo dado muestra que un modelo que produce un "chain of thought" puede resolver un problema.  Sin embargo, la información no describe la técnica en sí misma, solo indica su idoneidad para este propósito.




**RESPUESTA**: Al achicar el tamaño de los chunks se obtienen respuestas especificas pero distintas a lo que buscabamos. La respuesta es incompleta y difiere de lo que se esperaba. Esto se debe a que al achicar el tamaño de los chunks, se obtiene menos información y contexto para responder la pregunta.

In [ ]:
try_other_config(chunk_size=750)

Respuesta esperada:  Un Transformer es un tipo de arquitectura de red neuronal que usa un encoder-decoder para modelar relaciones en datos secuenciales.
Respuesta obtenita:  Un Transformer es una nueva arquitectura de red simple basada únicamente en mecanismos de atención, que prescinde por completo de la recurrencia y las convoluciones.  Está compuesta por un codificador y un decodificador, ambos con capas apiladas de auto-atención y capas totalmente conectadas puntuales.  El codificador tiene una pila de N=6 capas idénticas, cada una con dos subcapas: un mecanismo de auto-atención multi-cabeza y una red de avance totalmente conectada posicional simple.  Se utiliza una conexión residual alrededor de cada una de las dos subcapas, seguida de una normalización de capa.  Se aplica abandono (dropout) a la salida de cada subcapa, antes de que se agregue a la entrada de la subcapa y se normalice.  También se aplica abandono a las sumas de las incrustaciones y las codificaciones posicionales 

**RESPUESTA**: Al aumentar el tamaño de los chunks se obtiene información muchisimo más completa. Al tener tanta información, y dejar que el LLM la procese, se obtiene una respuesta mucho más completa y correcta.

**PROBANDO CAMBIAR CANTIDAD DE CHUNKS**

In [ ]:
try_other_config(ammount_of_chunks=1)

Respuesta esperada:  Un Transformer es un tipo de arquitectura de red neuronal que usa un encoder-decoder para modelar relaciones en datos secuenciales.
Respuesta obtenita:  Basado en la información proporcionada, un Transformer es un modelo de arquitectura que utiliza capas apiladas de auto-atención y capas totalmente conectadas puntuales, tanto para el codificador como para el decodificador.  El codificador está compuesto por una pila de N=6 capas idénticas, cada una con dos subcapas: una de mecanismo de auto-atención multi-cabeza y otra, una capa totalmente conectada simple y posicional.  La figura 1 muestra la arquitectura completa del modelo, con el codificador en la mitad izquierda y el decodificador en la mitad derecha.


Respuesta esperada:  Se puede descomponer el problema en partes más pequeñas y resolver cada parte por separado.
Respuesta obtenita:  Según la información proporcionada, Chain of Thought es una técnica que se puede usar para resolver problemas complejos con un 

**RESPUESTA**: Al disminuir la cantidad de chunks recuperados, se obtiene una respuesta más general y menos precisa, ya que el modelo obtieen menos contexto de diferentes partes del documento. Sin embargo, la respuesta en este caso es bastante buena, probablemente debido a que el tamaño de los chunks es lo suficientemente grande para que el modelo pueda responder correctamente, y que el chunk que se obtiene es realmente suficiente.

In [ ]:
try_other_config(ammount_of_chunks=8)

Respuesta esperada:  Un Transformer es un tipo de arquitectura de red neuronal que usa un encoder-decoder para modelar relaciones en datos secuenciales.
Respuesta obtenita:  El Transformer es una nueva y simple arquitectura de red propuesta como modelo de transducción de secuencias.  A diferencia de los modelos dominantes basados en redes neuronales recurrentes o convolucionales complejas, el Transformer se basa completamente en la auto-atención para calcular las representaciones de su entrada y salida, sin usar RNNs o convoluciones alineados con la secuencia.  Es el primer modelo de transducción que hace esto.  Su arquitectura utiliza capas apiladas de auto-atención y capas totalmente conectadas puntuales, tanto para el codificador como para el decodificador.  El codificador está compuesto por una pila de N=6 capas idénticas, cada una con dos subcapas: un mecanismo de auto-atención multi-cabeza y una capa de conexión totalmente conectada posicional.  El modelo también utiliza codifica

**RESPUESTA**: Al aumentar la cantidad de chunks recuperados, se obtiene una respuesta más completa y precisa, ya que el modelo obtiene más contexto de diferentes partes del documento. Esto se cumple en este caso, entregando una respuesta bastante completa.

**PROBANDO CAMBIAR TIPO DE BUSQUEDA**

In [ ]:
try_other_config(search_type='mmr')

Respuesta esperada:  Un Transformer es un tipo de arquitectura de red neuronal que usa un encoder-decoder para modelar relaciones en datos secuenciales.
Respuesta obtenita:  Un Transformer es un modelo de arquitectura que utiliza capas apiladas de auto-atención y capas totalmente conectadas punto a punto, tanto para el codificador como para el decodificador.  El codificador está compuesto por una pila de N=6 capas idénticas, cada una con dos subcapas: un mecanismo de auto-atención multi-cabeza y una capa de conexión totalmente conectada posicional.  Existen diferentes tamaños de Transformers,  por ejemplo, uno con (L=6, H=1024, A=16) y 100M parámetros, y otro más grande (L=64, H=512, A=2) con 235M parámetros.  Se ha demostrado que aumentar el tamaño del modelo lleva a mejoras continuas en tareas a gran escala como la traducción automática y el modelado del lenguaje.


Respuesta esperada:  Se puede descomponer el problema en partes más pequeñas y resolver cada parte por separado.
Respue

**RESPUESTA**: Al cambiar similarity por MMR (Maximal Marginal Relevance), se obtiene una respuesta más completa gracias a la diversidad de los chunks recuperados. Esto dado a que MMR busca chunks que sean diferentes entre sí, lo que permite que el modelo tenga más contexto para responder la pregunta.

### **2.2 Agentes (1.0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/rcqnN2aJCSEAAAAd/secret-agent-man.gif"
" width="400">
</p>

Similar a la sección anterior, en esta sección se busca habilitar **Agentes** para obtener información a través de tools y así responder la pregunta del usuario.

#### **2.2.1 Tool de Tavily (0.2 puntos)**

Generar una *tool* que pueda hacer consultas al motor de búsqueda **Tavily**.

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results = 1)

#### **2.2.2 Tool de Wikipedia (0.2 puntos)**

Generar una *tool* que pueda hacer consultas a **Wikipedia**.

*Hint: Le puede ser de ayuda el siguiente [link](https://python.langchain.com/v0.1/docs/modules/tools/).*

In [ ]:
%pip install wikipedia

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(lang='es', top_k_results=1, doc_content_chars_max=100)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

#### **2.2.3 Crear Agente (0.3 puntos)**

Crear un agente que pueda responder preguntas preguntas usando las *tools* antes generadas. Asegúrese que su agente responda en español. Por último, guarde el agente en una variable.

In [ ]:
from langchain import hub

react_prompt = hub.pull("hwchase17/react") # template de ReAct
print(react_prompt.template)

c:\Users\lucas\python-envs\nb-env\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
from langchain.agents import create_react_agent, AgentExecutor

tools = [tavily_tool, wiki_tool]

agent = create_react_agent(llm, tools, react_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'tavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.\nwikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.', 'tool_names': 'tavily_search_results_json, wikipedia'}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\

In [ ]:
def call_agent(input: str):
    response = agent_executor.invoke({"input": input})
    print(response["output"])

call_agent("Dame el nombre de algunos profesores de la FCFM de la Universidad de Chile")



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to find a list of professors from the Faculty of Physics and Mathematics (FCFM) at the University of Chile.  A Wikipedia search might be a good starting point, but it might not be completely up-to-date.

Action: wikipedia
Action Input: "Faculty of Physics and Mathematics, University of Chile"
Page: Universidad de California en Berkeley
Summary: La Universidad de California en Berkeley (en idThought: That was the wrong Wikipedia page. I need to try a different search term.  Let's try searching directly for the faculty's website.  If that doesn't work, I'll try a different approach.

Action: wikipedia
Action Input: "Facultad de Ciencias Físicas y Matemáticas, Universidad de Chile"
Page: Facultad de Ciencias Físicas y Matemáticas de la Universidad de Chile
Summary: La Facultad de Thought: The Wikipedia page for the Facultad de Ciencias Físicas y Matemáticas (FCFM) of the University of Chile should list facult

#### **2.2.4 Verificación de respuestas (0.3 puntos)**

Pruebe el funcionamiento de su agente y asegúrese que el agente esté ocupando correctamente las tools disponibles. ¿En qué casos el agente debería ocupar la tool de Tavily? ¿En qué casos debería ocupar la tool de Wikipedia?

In [ ]:
call_agent("Quién es Obama?")



> Entering new AgentExecutor chain...
Thought: I need to find information about Barack Obama.  Wikipedia is a good resource for biographical information.

Action: wikipedia
Action Input: Barack Obama
Page: Barack Obama
Summary: Barack Hussein Obama II[1]​ ( en inglés estadounidense; Honolulu, 4 de aQuestion: Quién es Obama?
Thought: I need to find information about Barack Obama.  Wikipedia is a good resource for biographical information.

Action: wikipedia
Action Input: Barack Obama
Page: Barack Obama
Summary: Barack Hussein Obama II[1]​ ( en inglés estadounidense; Honolulu, 4 de aQuestion: Quién es Obama?
Thought: I need to find information about Barack Obama. Wikipedia is a good resource for biographical information.
Action: wikipedia
Action Input: Barack ObamaPage: Barack Obama
Summary: Barack Hussein Obama II[1]​ ( en inglés estadounidense; Honolulu, 4 de aQuestion: Quién es Obama?
Thought: I need to find information about Barack Obama. Wikipedia is a good resource for biographic

In [ ]:
call_agent("Cómo le está yendo al FC Barcelona en la liga española?")



> Entering new AgentExecutor chain...
Thought: To answer this question about FC Barcelona's performance in the Spanish league, I need current standings and possibly recent results.  A search engine like tavily_search_results_json should provide up-to-date information.

Action: tavily_search_results_json
Action Input: "FC Barcelona La Liga standings"
[{'url': 'https://www.foxsports.com/soccer/barcelona-team-standings', 'content': 'Barcelona Laliga Standings | FOX Sports SPORTS & TEAMS SPORTS SPORTS & TEAMS SPORTS NFL NCAA FB MLB NBA NCAA BK NASCAR Soccer UFL NCAAW BK NHL Golf Premier Boxing Champions WWE UFC WNBA Tennis Motor Sports Professional Bowlers Association Horse Racing Westminster Kennel Club FIBA Olympics World Baseball Classic NRL The Basketball Tournament BARCELONA 9-0-1 · 1ST IN LALIGA · CHAMPIONS LEAGUE 9-0-1 · 1ST IN LALIGA · CHAMPIONS LEAGUE Next Match vs Real Madrid (A) · Sat 7:00PM BARCELONA LALIGA STANDINGS uefa champions league uefa europa league When games are liv

En general debería usar Wikipedia para información especifica que suele estar contenida en wikipedia, mientras que en cualquier otro caso más general debería usar Tavily. Que tipo de información debería buscar en wikipedia? Cuando se le pregunta el nombre de una persona, una entidad economica, etc. Es posible que incluso en esos casos se use la de Tavily, pero ese sería el criterio.

### **2.3 Multi Agente (1.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/r7QMJLxU4BoAAAAd/this-is-getting-out-of-hand-star-wars.gif"
" width="450">
</p>

El objetivo de esta subsección es encapsular las funcionalidades creadas en una solución multiagente con un **supervisor**.


#### **2.3.1 Generando Tools (0.5 puntos)**

Transforme la solución RAG de la sección 2.1 y el agente de la sección 2.2 a *tools* (una tool por cada uno).

In [ ]:
from langchain.tools import tool

@tool
def call_rag(question: str) -> str:
    """Call the RAG model to answer a question"""
    return rag_chain.invoke(question)

@tool
def call_react(input: str) -> str:
    """Call the ReAct model to answer a question"""
    return agent_executor.invoke({"input": input})["output"]

#### **2.3.2 Agente Supervisor (0.5 puntos)**

Habilite un agente que tenga acceso a las tools del punto anterior y pueda responder preguntas relacionadas. Almacene este agente en una variable llamada supervisor.

In [ ]:
supervisor_prompt = PromptTemplate.from_template(
    """
    Eres un agente enrrutador de preguntas.
    Tu rol es decidir que acción tomar para poder contestar la pregunta de la mejor manera posible:
    - 'llm': Cuando tengas que usar un sistema RAG para extraer información sobre LLMs. Esto incluye información de BERT, Transformers, CoT, etc.
    - 'general': Cuando la pregunta sea relacionada a cosas que puedan estar en internet o personas o entidades que conoce wikipedia.
    - 'otro': Todo aquella pregunta que no esté contenida en las categorías anteriores.
    - 'fin': Si crees que la pregunta ya puede ser contestada con la información actual.

    No respondas con más de una palabra y no incluyas.

    Pregunta:
    {question}


    Información actual:
    {context}


    Categoría:"""
)

supervisor_chain = (
    supervisor_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
answer_prompt = PromptTemplate.from_template(
    """
    Eres un agente experto en usar información dada para responder una pregunta.
    Tu rol es usar la información entregada para responder la pregunta de la mejor manera posible.
    No seas tan exigente con la información necesaria, si crees que puedes responder con la información entregada, hazlo.

    Información dada:
    {context}


    Pregunta:
    {question}


    Respuesta:"""
)

answer_chain = (
    answer_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
redirect_prompt = PromptTemplate.from_template(
    """
    Eres un asistente experto en el redireccionamiento de preguntas de usuarios.
    Vas a recibir una pregunta del usuario, tu único rol es indicar que no puedes responder su pregunta y redireccionar al usuario
    para que te pregunte sobre papers relacionados a LLMs, o sobre cualquier cosa que pueda estar en internet o personas o
    entidades que conoce wikipedia.

    Recuerda ser amable y cordial en tu respuesta.

    Pregunta: {question}
    Respuesta cordial:"""
)

redirect_chain = (
    redirect_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
def supervisor_question(question):
    '''
    Recibe una pregunta de usuario.
    Supervisar la pregunta, redirigir si se requiere más información o responder si es posible.
    '''

    actual_context = "No information available"
    action = supervisor_chain.invoke({"context": actual_context, "question": question})

    while "fin" not in action:
        if "llm" in action:
            print("Usando RAG para extraer información sobre LLMs")
            extra_info = call_rag(question)
        elif "general" in action:
            print("Usando ReAct para buscar información general")
            extra_info = call_react(question)
        else:
            print("No se pueden encontrar herramientas para responder la pregunta")
            action = redirect_chain.invoke({"question": question})
            return action
        actual_context = actual_context + "\n" + extra_info if actual_context != "No information available" else extra_info
        action = supervisor_chain.invoke({"context": actual_context, "question": question})
    print("Se posee toda la información necesaria para responder la pregunta")
    return answer_chain.invoke({"context": actual_context, "question": question})


#### **2.3.3 Verificación de respuestas (0.25 puntos)**

Pruebe el funcionamiento de su agente repitiendo las preguntas realizadas en las secciones 2.1.4 y 2.2.4 y comente sus resultados. ¿Cómo varían las respuestas bajo este enfoque?

In [ ]:
print(supervisor_question("Qué es BERT?"))

Usando RAG para extraer información sobre LLMs
Se posee toda la información necesaria para responder la pregunta
BERT es un modelo de lenguaje que alcanza resultados de vanguardia en once tareas de procesamiento del lenguaje natural.  Su arquitectura es unificada para diferentes tareas, permitiendo manejar varias tareas como respuesta a preguntas e inferencia de lenguaje sin modificaciones significativas en su arquitectura.  Es conceptualmente simple pero empíricamente poderoso, mostrando mejoras significativas en benchmarks como GLUE, MultiNLI y SQuAD v1.1.  Se inicializa con parámetros pre-entrenados y luego se ajusta finamente para tareas específicas usando datos etiquetados.  Puede representar tanto oraciones individuales como pares de oraciones (como pregunta-respuesta) en una secuencia de tokens.



In [ ]:
print(supervisor_question("Según Chain of Thought, ¿qué técnica se puede usar para resolver un problema complejo con un LLM?"))

Usando RAG para extraer información sobre LLMs
Usando RAG para extraer información sobre LLMs
Se posee toda la información necesaria para responder la pregunta
Según la información proporcionada, la técnica Chain of Thought (pensamiento en cadena) descompone problemas complejos en subproblemas más pequeños y manejables, imitando un proceso de pensamiento paso a paso para llegar a la respuesta.  Esto se logra, en parte,  proporcionando ejemplos de razonamiento de pensamiento en cadena en la indicación de pocos disparos (few-shot prompting) para que el modelo de lenguaje pueda generar sus propias cadenas de pensamiento.



In [ ]:
print(supervisor_question("Quién es Obama?"))

Usando ReAct para buscar información general


> Entering new AgentExecutor chain...
Thought: I need to find information about Barack Obama.  Wikipedia is a good resource for biographical information.

Action: wikipedia
Action Input: Barack Obama
Page: Barack Obama
Summary: Barack Hussein Obama II[1]​ ( en inglés estadounidense; Honolulu, 4 de aQuestion: Quién es Obama?
Thought: I need to find information about Barack Obama.  Wikipedia is a good resource for biographical information.

Action: wikipedia
Action Input: Barack Obama
Page: Barack Obama
Summary: Barack Hussein Obama II[1]​ ( en inglés estadounidense; Honolulu, 4 de aQuestion: Quién es Obama?
Thought: I need to find information about Barack Obama. Wikipedia is a good resource for biographical information.
Action: wikipedia
Action Input: Barack ObamaPage: Barack Obama
Summary: Barack Hussein Obama II[1]​ ( en inglés estadounidense; Honolulu, 4 de aQuestion: Quién es Obama?
Thought: I need to find information about Barack Obama

In [ ]:
print(supervisor_question("Cómo le está yendo al FC Barcelona en la liga española?"))

Usando ReAct para buscar información general


> Entering new AgentExecutor chain...
Thought: To answer this question about FC Barcelona's performance in the Spanish league, I need current standings and possibly recent results.  A search engine like tavily_search_results_json should provide this information.

Action: tavily_search_results_json
Action Input: "FC Barcelona La Liga standings 2023-2024"
[{'url': 'https://www.eurosport.com/football/la-liga/2023-2024/standings.shtml', 'content': 'Stay up to date with the 2023/2024 La Liga table. Follow the top teams and make Eurosport your go-to source for Football tables and results. ... FC Barcelona BAR. 38: 26: 7: 5: 79: 44: 35: 85: 3'}]Thought: The observation shows a snippet indicating FC Barcelona's position in La Liga. However, it's a small snippet and might not be entirely up-to-date. To get a more complete and reliable picture, I should consult a dedicated sports website or news source.  I'll try a different search.

Action: tavily_

**RESPUESTA**: Las respuestas son bastante similares a las entregadas por los agentes individuales, sin embargo en algunas corridas las respuestas mejoran, ya que al detectar una respuesta que no es lo suficientemente buena, o que no tiene la información que se busca, vuelve a buscar ya sea en esa fuente o en otra. Esto permite que las respuestas sean correctas en la mayoría de los casos.

#### **2.3.4 Análisis (0.25 puntos)**

¿Qué diferencias tiene este enfoque con la solución *Router* vista en clases? Nombre al menos una ventaja y desventaja.

**RESPUESTA**: La diferencia principal es que al correr es cómo se maneja la respuesta entregada por las tools que creamos. El router por un lado solo redirige la pregunta a donde corresponda, mientras que el supervisor puede volver a buscar en otra fuente y evaluar la respuesta es correcta o si es lo que se pide. Con esto se logra que las respuestas sean más precisas y correctas. Sin embargo, esto también puede ser una desventaja, ya que va a implicar un costo y tiempo adicional por la forma de procesar las respuestas.

### **2.4 Memoria (Bonus +0.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/Gs95aiElrscAAAAd/memory-unlocked-ratatouille-critic.gif"
" width="400">
</p>

Una de las principales falencias de las soluciones que hemos visto hasta ahora es que nuestro chat no responde las interacciones anteriores, por ejemplo:

- Pregunta 1: "Hola! mi nombre es Sebastián"
  - Respuesta esperada: "Hola Sebastián! ..."
- Pregunta 2: "Cual es mi nombre?"
  - Respuesta actual: "Lo siento pero no conozco tu nombre :("
  - **Respuesta esperada: "Tu nombre es Sebastián"**

Para solucionar esto, se les solicita agregar un componente de **memoria** a la solución entregada en el punto 2.3.

**Nota: El Bonus es válido <u>sólo para la sección 2 de Large Language Models.</u>**

### **2.5 Despliegue (0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/IytHqOp52EsAAAAd/you-get-a-deploy-deploy.gif"
" width="400">
</p>

Una vez tengan los puntos anteriores finalizados, toca la etapa de dar a conocer lo que hicimos! Para eso, vamos a desplegar nuestro modelo a través de `gradio`, una librería especializada en el levantamiento rápido de demos basadas en ML.

Primero instalamos la librería:

In [ ]:
%pip install --upgrade --quiet gradio

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-chroma 0.1.1 requires langchain-core<0.3,>=0.1.40, but you have langchain-core 0.3.19 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Luego sólo deben ejecutar el siguiente código e interactuar con la interfaz a través del notebook o del link generado:

In [ ]:
import gradio as gr
import time

def agent_response(message, history):
  '''
  Función para gradio, recibe mensaje e historial, devuelte la respuesta del chatbot.
  '''
  # get chatbot response
  response = ... # rellenar con la respuesta de su chat

  # assert
  assert type(response) == str, "output de route_question debe ser string"

  # "streaming" response
  for i in range(len(response)):
    time.sleep(0.015)
    yield response[: i+1]

gr.ChatInterface(
    agent_response,
    type="messages",
    title="Chatbot MDS7202", # Pueden cambiar esto si lo desean
    description="Hola! Soy un chatbot muy útil :)", # también la descripción
    theme="soft",
    ).launch(
        share=True, # pueden compartir el link a sus amig@s para que interactuen con su chat!
        debug = False,
        )

KeyboardInterrupt: 